In [1]:
import json
import uuid
import os

In [6]:
class IdManager:
    def __init__(self, id_file="item_ids.json"):
        self.id_file = id_file
        self.name_to_id = {}
        self.load_ids()

    def load_ids(self):
        """Load existing IDs from file if it exists"""
        if os.path.exists(self.id_file):
            with open(self.id_file, 'r') as f:
                self.name_to_id = json.load(f)

    def save_ids(self):
        """Save IDs to file"""
        with open(self.id_file, 'w') as f:
            json.dump(self.name_to_id, f, indent=2)

    def get_id(self, item_name):
        """Get existing ID or generate new one for an item"""
        if item_name not in self.name_to_id:
            self.name_to_id[item_name] = str(uuid.uuid4()).replace("-", "")[:20]
            self.save_ids()
        return self.name_to_id[item_name]

def convert_item(item, id_manager):
    """Convert a single Genesys item to Questline VTT format."""
    vtt_item = {
        "actions": [],
        "type": "genesys",
        "version": 1,
        "name": item.get("name", ""),
        "creator": "SYSTEM",
        "description": "",
        "privacy": {
            "users": [],
            "level": "public"
        },
        "data": {
            "type": item.get("type", ""),
            "subtype": "",
            "rarity": item.get("rarity", 0),
            "skill": "",
            "damage": "",
            "critical": "",
            "range": "",
            "cost": {
                "gold": item.get("price", 0)
            },
            "encumbrance": item.get("encumbrance", 0),
            "defense": "",
            "soak": "",
            "special": ""
        },
        # Use IdManager to get consistent ID
        "id": id_manager.get_id(item.get("name", "")),
        "uses": {
            "max": 0
        }
    }

    # Handle description
    if isinstance(item.get("description"), list):
        vtt_item["description"] = " ".join(
            [d if isinstance(d, str) else "" for d in item["description"]]
        )
    elif isinstance(item.get("description"), str):
        vtt_item["description"] = item["description"]

    # Handle weapon-specific properties
    if item.get("type") == "weapon":
        vtt_item["data"].update({
            "damage": str(item.get("damage", "")),
            "critical": str(item.get("critical", "")),
            "range": str(item.get("range", "")),
            "skill": item.get("skill", {}).get("name", "")
        })

    # Handle armor-specific properties
    if item.get("type") == "armor":
        vtt_item["data"].update({
            "defense": str(item.get("defense", "")),
            "soak": str(item.get("soak", ""))
        })

    # Handle special qualities
    if "qualities" in item:
        special_qualities = []
        for quality in item["qualities"]:
            quality_text = f"{quality['name']}"
            if "ranks" in quality:
                quality_text += f" {quality['ranks']}"
            special_qualities.append(quality_text)
        vtt_item["data"]["special"] = ", ".join(special_qualities)

    return vtt_item


def convert_genesys_to_vtt(input_file):
    """Convert Genesys JSON file to Questline VTT format."""
    # Create ID manager instance
    id_manager = IdManager(id_file=f"{input_file.replace('.json', '-item-ids')}.json")

    with open(input_file, 'r', encoding='utf-8') as f:
        source_data = json.load(f)

    # Extract items from all possible categories
    categories = ["gear", "weapons", "armor", "implements"]
    all_items = []
    for category in categories:
        if category in source_data:
            all_items.extend(source_data[category])

    # Convert all items using ID manager
    vtt_items = [convert_item(item, id_manager) for item in all_items]

    # Save to output file
    with open(input_file.replace('.json', '.items'), 'w', encoding='utf-8') as f:
        json.dump(vtt_items, f, indent=2)


if __name__ == "__main__":
    convert_genesys_to_vtt("core-rule-book.json")